In [ ]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

In [4]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Incorrect setup.")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import json

DATASET_PATH = "/content/drive/My Drive/alpaca_final.json"

with open(DATASET_PATH, "r", encoding="utf-8") as f:
  dataset = json.load(f)

!cp "/content/drive/My Drive/alpaca_final.json" "/content/LLaMA-Factory/data/alpaca_final.json"

In [ ]:
!huggingface-cli login

In [28]:
import json

with open("/content/LLaMA-Factory/data/dataset_info.json", "r", encoding="utf-8") as f:
    dataset_info = json.load(f)

dataset_info["alpaca_final"] = {
    "file_name": "alpaca_final.json",
    "language": "ja",
    "task": "instruction",
    "license": "unknown"
}

with open("/content/LLaMA-Factory/data/dataset_info.json", "w", encoding="utf-8") as f:
    json.dump(dataset_info, f, indent=2)

In [ ]:
import json

args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="meta-llama/Llama-3.2-3B-Instruct",
  dataset="identity,alpaca_final",
  template="llama3",
  finetuning_type="lora",
  lora_target="all",
  lora_rank=512,
  output_dir="llama3_lora",
  per_device_train_batch_size=16,
  gradient_accumulation_steps=1,
  lr_scheduler_type="cosine",
  logging_steps=10,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_samples=float('inf'),
  max_grad_norm=1.0,
  loraplus_lr_ratio=16.0,
  fp16=True,
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_llama3.json

In [ ]:
import json

args = dict(
  model_name_or_path="meta-llama/Llama-3.2-3B-Instruct",
  adapter_name_or_path="llama3_lora",
  template="llama3",
  finetuning_type="lora",
  export_dir="llama3_lora_merged",
  export_size=2,
  export_device="cpu",
)

json.dump(args, open("merge_llama3.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli export merge_llama3.json

In [ ]:
%cd /content/LLaMA-Factory/llama3_lora_merged

In [ ]:
ls

In [42]:
!cp tokenizer.json /content/drive/MyDrive/